Actividad 1 

Para el preprocesamiento de datos, llevamos a cabo una limpieza de columnas y rellenamos los datos perdidos con ceros, esta última decisión la tomamos al ser que la columna INS_1SEM e INS_2SEM proporciona información de las asignaturas que se tomaron por cada semestre cada alumno, lo que nos llevó a conjeturar que los promedios que se encontraban vacíos es porque el estudiante no los inscribió por lo que nos pareció más prudente reemplazarlos por ceros que por la media, la cual era otra de nuestras opciones.
A continuación mencionamos las columnas que determinamos que no son relevantes para este objetivo del taller: ACTF_1SEM_R, ACTF_2SEM_R, descripcion_situacion_egreso_postulante, nombre_secretaria_admision, descripcion_jefe_familia, descripcion_nivel_educacion_padre, descripcion_nivel_educacion_madre, descipcion_tipo_organismo_trabajan_padre, descripcion_tipo_organismo_trabajan_madre, descripcion_ocupacion_principal_padre, descripcion_ocupacion_principal_madre, descripcion_rama_actividad_padre, descripcion_rama_actividad_madre, cuantos_trabajan_grupos_familiar, cuantos_estudian_grupo_familiar, cuantos_estudian_grupo_pre_basica, cuantos_estudian_grupo_media_1_3, cuantos_estudian_grupo_media_4, cuantos_estudian_grupo_otras.


Actividad 2 

En este caso se usó <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html">KFold</a>. Recuerde que también existe <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html">train_test_split</a>.


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

#Cargar los datos desde el archivo CSV
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

#Separar las características y la variable objetivo
X = datos.iloc[:, :-1]
y = datos.iloc[:, -1]

#Eliminar la columna "Id"
X = X.drop('Id', axis=1)

#Convertir variables categóricas en dummies
X = pd.get_dummies(X)

#Convertir las etiquetas categóricas en valores numéricos
y = pd.get_dummies(y)

#Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

'''
Mostrar las primeras filas de los conjuntos de datos
print("Conjunto de entrenamiento:")
display(X_train.head())
display(y_train.head())

print("\nConjunto de prueba:")
display(X_test.head())
display(y_test.head())

Crear el modelo de red neuronal
'''

model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))  # Capa de entrada
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Capa de salida con dos neuronas para clasificación binaria con función softmax

#Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

#Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f'Pérdida en el conjunto de prueba: {loss:.4f}')
print(f'Precisión en el conjunto de prueba: {accuracy:.4f}')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

Actividad 3

En este caso debieran que ser 4 bloques 2 modelos y dos modificaciones de parametros para cada modelo 

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier
from scipy.stats import uniform, randint

# Cargar los datos desde el archivo CSV
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Ajustar los nombres de las columnas
datos.rename(columns={'Rotulo_Titulado': 'Rotulo_Titulo', 'Rotulo_Titulo_Si': 'Rotulo_Titulo_SI'}, inplace=True)

# Separar las características y la variable objetivo
X = datos.drop(['Id', 'Rotulo_Titulo'], axis=1)  # Eliminar 'Id' y 'Rotulo_Titulo' del conjunto de características
y = datos['Rotulo_Titulo']  # Variable objetivo

# Convertir variables categóricas en dummies
X = pd.get_dummies(X)

# Convertir las etiquetas categóricas en valores numéricos
y = pd.get_dummies(y, drop_first=True)  # Convertir a numérico y evitar la trampa de la variable ficticia

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertir los datos a tipo float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Definir una función para crear el modelo Keras
def create_model(neurons=32, dropout_rate=0.5, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Crear el clasificador Keras
model = KerasClassifier(model=create_model, verbose=0)

# Definir la distribución de parámetros para la búsqueda aleatoria
param_dist = {
    'model__neurons': randint(16, 128),
    'model__dropout_rate': uniform(0.1, 0.5),
    'model__learning_rate': uniform(0.0001, 0.01),
    'batch_size': [16, 32, 64],
    'epochs': [50, 100]
}

# Realizar la búsqueda aleatoria
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)
random_search_result = random_search.fit(X_train, y_train)

# Imprimir los mejores parámetros y la mejor precisión
print(f'Mejor: {random_search_result.best_score_} usando {random_search_result.best_params_}')

# Evaluar el mejor modelo en el conjunto de prueba
best_model = random_search_result.best_estimator_.model
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Pérdida en el conjunto de prueba: {loss:.4f}')
print(f'Precisión en el conjunto de prueba: {accuracy:.4f}')


Fitting 3 folds for each of 10 candidates, totalling 30 fits


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Mejor: 0.705051862839014 usando {'batch_size': 32, 'epochs': 100, 'model__dropout_rate': 0.29270825126995803, 'model__learning_rate': 0.00025966252220214196, 'model__neurons': 17}


AttributeError: 'function' object has no attribute 'evaluate'

Actividad 4

Aplicacion del modelo a nuevo conjunto

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load the data
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Separate features and target variable
X = datos.iloc[:, :-1]
y = datos.iloc[:, -1]

# Drop the "Id" column
X = X.drop('Id', axis=1)

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Create preprocessing pipelines for numerical and categorical data
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
    ('scaler', StandardScaler())  # Scale features
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with the most frequent value
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Combine preprocessing steps
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# Preprocess the data
X_processed = preprocessor.fit_transform(X)

# Convert target variable to numerical (binary encoding)
y = pd.get_dummies(y, drop_first=True).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Convert to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Build the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))  # Input layer
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(32, activation='relu'))  # Hidden layer 1
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(16, activation='relu'))  # Hidden layer 2
model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5111 - loss: 0.7262 - val_accuracy: 0.6915 - val_loss: 0.5909
Epoch 2/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6527 - loss: 0.6065 - val_accuracy: 0.7101 - val_loss: 0.5267
Epoch 3/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6718 - loss: 0.5809 - val_accuracy: 0.7164 - val_loss: 0.5131
Epoch 4/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7194 - loss: 0.5543 - val_accuracy: 0.7184 - val_loss: 0.5150
Epoch 5/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7252 - loss: 0.5408 - val_accuracy: 0.7267 - val_loss: 0.5105
Epoch 6/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7255 - loss: 0.5438 - val_accuracy: 0.7226 - val_loss: 0.5132
Epoch 7/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7333 - loss: 0.5556 - val_accuracy: 0.7267 - val_loss: 0.5082
Epoch 8/50
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7586 - loss: 0.5272 - val_accuracy: 0.7308 - val_loss

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load the data
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Separate features and target variable
X = datos.iloc[:, :-1]
y = datos.iloc[:, -1]

# Drop the "Id" column
X = X.drop('Id', axis=1)

# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Create preprocessing pipelines for numerical and categorical data
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Change to median imputation
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# Preprocess the data
X_processed = preprocessor.fit_transform(X)

# Convert target variable to numerical (binary encoding)
y = pd.get_dummies(y, drop_first=True).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Convert to float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Build the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.4))  # Adjust dropout rate
model.add(Dense(128, activation='relu'))  # Increase layer complexity
model.add(Dropout(0.4))  # Adjust dropout rate
model.add(Dense(64, activation='relu'))  # Add another hidden layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')


Epoch 1/50


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [145]:
import pandas as pd

# Cargar los datos de evaluación
datos_evaluacion = pd.read_csv('Taller_2_Titulacion_Evaluación.csv', encoding='latin-1', delimiter=';')

# Eliminar la columna "Id" si es necesario
X_evaluacion = datos_evaluacion.drop('Id', axis=1)

# Convertir variables categóricas en dummies si es necesario
X_evaluacion = pd.get_dummies(X_evaluacion)

# Hacer predicciones con el modelo cargado
predictions = model.predict(X_evaluacion)

# Convertir las predicciones a etiquetas legibles
predicted_labels = np.where(predictions[:, 1], 'SÍ', 'NO')  # Si la segunda columna (índice 1) es True, entonces 'SÍ', de lo contrario 'NO'

# Crear un DataFrame con las columnas Id y Prediccion
resultados = pd.DataFrame({'Id': datos_evaluacion['Id'], 'Prediccion': predicted_labels})

# Guardar el DataFrame como un archivo de texto separado por comas
resultados.to_csv('clasificacion_titulo.txt', index=False, sep=',', header=True)


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- nombre_comuna_EM_CALBUCO
- nombre_comuna_EM_CERRO NAVIA
- nombre_comuna_EM_FREIRINA
- nombre_comuna_EM_LAMPA
- nombre_comuna_EM_PAILLACO
- ...
Feature names seen at fit time, yet now missing:
- descripcion_nivel_educacion_madre_Instituto profesional completa
- descripcion_nivel_educacion_madre_Sin estudios
- descripcion_ocupacion_principal_madre_Obrero agrícola o minero calificado& mecánico& electricista&  tractorista& etc.
- descripcion_rama_actividad_madre_Explotación de minas y canteras
- descripcion_rama_educacional_EM_Sin Datos
- ...
